# Satelliten-Wasserstände von Seen

Dieses Datenset liefert Wasserstandsmessungen für 251 Seen auf vier Kontinenten, abgeleitet aus satellitenbasierter Radaraltimetrie. Diese Messungen stellen eine kostengünstige Alternative zu traditionellen bodengestützten Systemen dar und werden als wesentliche Klimavariable (Essential Climate Variable, ECV) eingestuft. Die Daten werden von CLS für den Copernicus Climate Change Service erstellt, um die hydrologische und klimatische Überwachung auf kontinentaler Ebene zu verbessern.

**Informationen zum Datensatz:**
* Quelle: [Satellite Lake Water Levels](https://cds.climate.copernicus.eu/datasets/satellite-lake-water-level?tab=overview)
* Autor: T. Tewes (Stadt Konstanz)
* Notebook-Version: 1.4 (Aktualisiert: 17. Januar 2025)

## 1. Festlegen der Pfade und Arbeitsverzeichnisse

In [1]:
import os

''' ---- Verzeichnisse hier angeben ---- '''
download_folder = r".\data\satellite-lake-water-level\download"
working_folder = r".\data\satellite-lake-water-level\working"
geotiff_folder = r".\data\satellite-lake-water-level\geotiff"
csv_folder = r".\data\satellite-lake-water-level\csv"
output_folder = r".\data\satellite-lake-water-level\output"
''' ----- Ende der Angaben ---- '''

os.makedirs(download_folder, exist_ok=True)
os.makedirs(working_folder, exist_ok=True)
os.makedirs(geotiff_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)


## 2. Herunterladen und Entpacken des Datensatzes


### 2.1 Authentifizierung

In [2]:
import cdsapi

def main():
    # API-Schlüssel für die Authentifizierung
    api_key = "fdae60fd-35d4-436f-825c-c63fedab94a4"
    api_url = "https://cds.climate.copernicus.eu/api"
    
    # Erstellung des CDS-API-Clients
    client = cdsapi.Client(url=api_url, key=api_key)
    return client

### 2.2 Anforderungsdefinition und Download der Daten

In [3]:
# Definition des Datensatzes und der Anfrageparameter
dataset = "satellite-lake-water-level"
request = {
    "variable": "all", 
    "region": ["southern_europe"],
    "lake": ["bodensee"]
}

In [4]:
# Kommentiere diese Zelle ein und führe sie aus, um den Datensatz herunterzuladen:

def main_retrieve():
    dataset_filename = f"{dataset}_bodensee.zip"
    dataset_filepath = os.path.join(download_folder, dataset_filename)

    # Den Datensatz nur herunterladen, wenn er noch nicht heruntergeladen wurde
    if not os.path.isfile(dataset_filepath):
        # Den Datensatz mit den definierten Anforderungsparametern herunterladen
        client.retrieve(dataset, request, dataset_filepath)
    else:
        print("Datensatz bereits heruntergeladen.")

if __name__ == "__main__":
    client = main()
    main_retrieve()

2025-01-17 11:00:37,410 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.


2025-01-17 11:00:37,411 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


2025-01-17 11:00:37,412 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**


2025-01-17 11:00:37,412 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Datensatz bereits heruntergeladen.


### 2.3 Entpacken des Zip-Ordners

In [5]:
import zipfile

# Erstellen des Dateinamens und des Dateipfads für die ZIP-Datei des Datensatzes
dataset_filename = f"{dataset}_bodensee.zip"
dataset_filepath = os.path.join(download_folder, dataset_filename)
extract_folder = working_folder

# Entpacken der ZIP-Datei
try:
    if not os.listdir(extract_folder):
        with zipfile.ZipFile(dataset_filepath, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
            print(f"Dateien erfolgreich extrahiert nach: {extract_folder}")
    else:
        print("Ordner ist nicht leer. Entpacken überspringen.")
except FileNotFoundError:
    print(f"Fehler: Die Datei {dataset_filepath} wurde nicht gefunden.")
except zipfile.BadZipFile:
    print(f"Fehler: Die Datei {dataset_filepath} ist keine gültige ZIP-Datei.")
except Exception as e:
    print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")

Ordner ist nicht leer. Entpacken überspringen.


## 3. Lesen der netCDF4-Datei und Ausdrucken der Metadaten

In [6]:
import netCDF4 as nc

# Öffnen der NetCDF-Datei im Lesemodus
nc_filepath = os.path.join(extract_folder, os.listdir(extract_folder)[0])
dataset = nc.Dataset(nc_filepath, mode='r')

# Auflisten aller Variablen im Datensatz
variables_list = dataset.variables.keys()
print(f"Verfügbare Variablen: {list(variables_list)}")

Verfügbare Variablen: ['time', 'lat', 'lon', 'water_surface_height_above_reference_datum', 'water_surface_height_uncertainty']


In [7]:
import pandas as pd

variable_name = 'water_surface_height_above_reference_datum'
variable_data = dataset[variable_name]

# Erstellen einer Zusammenfassung der Hauptvariablen
summary = {
    "Variablenname": variable_name,
    "Daten Typ": variable_data.dtype,
    "Form": variable_data.shape,
    "Variableninfo": f"{variable_name}({', '.join(variable_data.dimensions)})",
    "Einheiten": getattr(variable_data, "units", "N/A"),
    "Langer Name": getattr(variable_data, "long_name", "N/A"),
}

# Anzeigen der Zusammenfassung des Datensatzes als DataFrame zur besseren Visualisierung
nc_summary = pd.DataFrame(list(summary.items()), columns=['Beschreibung', 'Details'])

# Anzeigen des Zusammenfassungs-DataFrames
nc_summary

,Beschreibung,Details
0,Variablenname,water_surface_height_above_reference_datum
1,Daten Typ,float64
2,Form,"(101,)"
3,Variableninfo,water_surface_height_above_reference_datum(time)
4,Einheiten,m
5,Langer Name,water surface height above geoid


## 4. Exportieren als CSV

### 4.1 Erstellen des DataFrames und Exportieren als CSV

In [8]:
import netCDF4 as nc

# Funktion zum Umwandeln von netCDF in CSV (nur Zeit und eine Variable)
def netcdf_to_dataframe(nc_file):
    with nc.Dataset(nc_file, mode="r") as nc_dataset:
        # Extrahiere die Daten und die Zeit
        time = nc_dataset.variables['time'][:]
        time_units = nc_dataset.variables['time'].units
        time_calendar = getattr(nc_dataset.variables['time'], 'calendar', 'standard')
        cftime = nc.num2date(time, units=time_units, calendar=time_calendar)
        
        # Extrahiere die Daten
        water_surface_height = nc_dataset.variables['water_surface_height_above_reference_datum'][:]
        wsh_units = nc_dataset.variables["water_surface_height_above_reference_datum"].units
        wsh_uncertainty = nc_dataset.variables['water_surface_height_uncertainty'][:]
        
        # Erstelle eine Liste von Einträgen für das DataFrame (nur Zeit und die Variable)
        df = pd.DataFrame(
            {
                "Zeit": cftime,
                f"Wasserstandshöhe ({wsh_units})": water_surface_height,
                f"Unsicherheit": wsh_uncertainty,
            }
        )
       
        # Setze den Index auf die Zeit
        return df.set_index('Zeit')

# Rufe die Funktion auf und erhalte das DataFrame
df = netcdf_to_dataframe(nc_filepath)

# Generiere den Ausgabe-Dateipfad
csv_filename = 'satellite-lake-water-level.csv'
csv_path = os.path.join(csv_folder, csv_filename)

# Exportiere die Pivot-Tabelle als CSV, falls sie noch nicht existiert
if not os.path.isfile(csv_path):
    df.to_csv(csv_path)
    print(f"Zusammenfassungsdaten wurden nach {csv_path} exportiert.")
else:
    print(f"Die Datei {csv_path} existiert bereits. Export wird übersprungen.")

# Zeige das DataFrame an
df.head()

Die Datei .\data\satellite-lake-water-level\csv\satellite-lake-water-level.csv existiert bereits. Export wird übersprungen.


,Wasserstandshöhe (m),Unsicherheit
Zeit,,
2016-03-23 20:43:59.997482,394.44,0.29
2016-04-19 20:43:59.997482,394.62,0.49
2016-05-16 20:44:59.998284,395.41,0.21
2016-06-12 20:44:59.998284,395.72,0.58
2016-07-09 20:44:59.998284,396.13,0.13
